## Algorithms for DNA Sequencing Labs
## Week 4
    4.1) shortest common superstring
    4.2) greedy scs
    4.3) DeBruijn

## 4.1 shortest common superstring

In [161]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [168]:
t = readGenome('chr1.GRCh38.excerpt.fasta')

In [169]:
def editDistance(x, y):
    """Returns the edit distance between two strings, x and y"""
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]

In [170]:
def bestApproximateMatchEditDistance(p, t):
    """Returns the edit distance between two strings, p and t"""
    # Create distance matrix
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    
    # Initialize first row and column of matrix
    for i in range(len(p)+1):
        D[i][0] = i
    # See slide 4 on  0440_approx__editdist3.pdf
    # First row is already initialised to zero so we simply just comment the following two lines.
    #for i in range(len(p)+1):
    #    D[0][i] = i
    
    # Fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if p[i-1] == t[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)

    # Best Approximate Match Distance is the smallest value of the last row
    return min(D[-1])

def test1():
    """P = GCGTATGC within T = TATTGGCTATACGGTT had 2 edits."""
    assert bestApproximateMatchEditDistance('GCGTATGC', 'TATTGGCTATACGGTT') == 2

In [172]:
print(bestApproximateMatchEditDistance('GCTGATCGATCGTACG', t))

3


In [173]:
print(bestApproximateMatchEditDistance('GATTTACCAGATTGAG', t))

2


In [174]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching a prefix of 'b' that is at least 'min_length' characters long.  If no such overlap exists, return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [175]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

--2020-12-09 11:51:54--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.226.211.176, 13.226.211.135, 13.226.211.37, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.226.211.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  5.17MB/s    in 0.5s    

2020-12-09 11:51:55 (5.17 MB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [44]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [192]:
readFastq('ERR266411_1.for_asm.fastq')

(['...',
  ...],
 ['B@DFEFFFGEGGGHEHGHGHGGGGHIFGFIFHICFGHGHGJGHFGHGIHEHGGHJGFEFHGHEGGHHGHIFGFGDIFGGFGGGFHGGGHGGGAGIFGGCG',
  "=@@CEBF@BGBG6GF1E<04C3E7E.9G=H<H:HH;HBH;G69F7,7DG((EG8A8,-5-8,?,,,AE>C,B?,,,64$AC'+'='+4'3*4+E322*'(",
  'A@DFEFFFGDGGGHFHHHGHGGFGHILGFGEHHHFHHFIGHGHFGHGGGEHGGHJGGHFGFGHHEHFGHICKFGJEIGGIFGGGHGGFHIEEGGEFGKGD',
  'B@DFEHFEGEGGGHGFGHGHFFGGGI@GDFHHFFHGHGGFHCHFGHCIGEH9CHGEFGFHFDHEIHCGFGFDDCCGIDGIGKGGHGCFGIFGG;HFGFIE',
  'B@DFEDFEGGGGGFFHEHGIGGGGFIGGHHFHHHHGHGFGHHHFGEGFGGHGGHJGGHFHGEHH,HGGHFHGFAGEHHGIGCGGFGEFHFFGGDGFEGGD',
  '@@DFE9DFGEGDGD?HGHGBFCFEEDE?DFDDGG97BFHB6GEDA@GGGEHEGAHCEGFFAEFHEHFGHFEK?DJEHDGIGDGGHGGGJF@GFFE9C2GE',
  'B@DFEEFEGIGGGGGGGHGHGGGGFIFGGIFHFHHEHFHGHGHFGEGGHG8EGHGGHGFGIGHGIHGEFFHFGGJJIIGIGHGGHGGFJEGGGFGFGGGE',
  'B@DFEFFEGGGGGHFHGHGGGGGGFIGGFFEHGHFGHBFGGDHFG@GIGFHEGHGGHJFHGBHFGHGGHEHFFGJIEGGIGGGGFDCEFIGEGFFFGGGE',
  '8@@C@BFDBHBGB6F>A<GFCEH@F>H@EGFHBEFH?B9FDGA9GC@I@88D7H,8G9BH8E?6IF?6F?IB?6>I,H,5>C4G=:=A=3D44FB?DG+9',
  'B@DFFDFGGEGGGDGEGHGHGGFGHI

In [195]:
reads, qualities = readFastq('ERR266411_1.for_asm.fastq')

In [181]:
def overlap_all_pairs(reads, min_length):
    from pprint import pprint
    
    overlap_map = {}
    overlap_graph = {}
    overlap_pairs = []

    #We use a Python dictionary to associate each k-mer with its corresponding set. 
    suffixDict = {}
    for read in reads:
        kmers = getkmers(read, min_length)
        #print(kmers)
        #(1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer.
        for kmer in kmers:
            if not kmer in suffixDict.keys():
                #Let every k-mer in the dataset have an associated Python set object, which starts out empty. 
                suffixDict[kmer] = set()
            suffixDict[kmer].add(read)
    #pprint(suffixDict)
    
    #(2) Now, for each read a, we find all overlaps involving a suffix of a
    for read in reads:
        #we take a's length-k suffix,
        suffix = read[-min_length:]
        #if len(suffix) < min_length:
        #    continue

        #find all reads containing that k-mer (obtained from the corresponding set) ...
        matching_reads = suffixDict[suffix]

        #...and call overlap(a, b, min_length=k) for each.
        for read2 in matching_reads:
            # The most important point is that we do not call overlap(a, b, min_length=k) if b does not contain the length-k suffix of a.
            #if read2.find(suffix) >= 0 and read2 != suffix :
            if read2 != read :
                val = overlap(read, read2, min_length) 
                if val > 0:
                    overlap_map[ (read, read2) ] = val
                    overlap_graph[read] = read2
                    overlap_pairs.append( (read,read2) )

    #pprint(overlap_map)
    #pprint(overlap_pairs)
    #pprint(overlap_graph)
    return overlap_pairs, overlap_map, overlap_graph

In [182]:
def getkmers(read, kmer_length):
    """our read is GATTA and k=3, we would add GATTA to the set objects for GAT, ATT and TTA."""
    return [ read[i:i+kmer_length] for i in range(len(read)+1-kmer_length) ]

def test2():
    kmers = getkmers('GATTA', 3)
    assert kmers == ['GAT', 'ATT', 'TTA']

def example1():
    reads = ['ABCDEFG', 'EFGHIJ', 'HIJABC']
    assert overlap_all_pairs(reads, 3)[0] == [('ABCDEFG', 'EFGHIJ'), ('EFGHIJ', 'HIJABC'), ('HIJABC', 'ABCDEFG')]
    assert overlap_all_pairs(reads, 4)[0] == []

def example2():
    from pprint import pprint

    reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']

    results4, overlap_map4, overlap_graph4 = overlap_all_pairs(reads, 4)
    expected4 = [('CGTACG', 'TACGTA'),
                 ('CGTACG', 'GTACGT'),
                 ('CGTACG', 'GTACGA'),
                 ('CGTACG', 'TACGAT'),
                 ('TACGTA', 'ACGTAC'),
                 ('TACGTA', 'CGTACG'),
                 ('GTACGT', 'TACGTA'),
                 ('GTACGT', 'ACGTAC'),
                 ('ACGTAC', 'GTACGA'),
                 ('ACGTAC', 'GTACGT'),
                 ('ACGTAC', 'CGTACG'),
                 ('GTACGA', 'TACGAT')]

    assert sorted(results4) ==  sorted(expected4) , "example2, first assert failed"

    results5, overlap_map5, overlap_graph5 = overlap_all_pairs(reads, 5)
    expected5 = [('CGTACG', 'GTACGT'),
                 ('CGTACG', 'GTACGA'),
                 ('TACGTA', 'ACGTAC'),
                 ('GTACGT', 'TACGTA'),
                 ('ACGTAC', 'CGTACG'),
                 ('GTACGA', 'TACGAT')] 

    assert sorted(results5) ==  sorted(expected5), "example2, second assert failed"

In [187]:
def question3and4(reads):
    from pprint import pprint

    overlap_pairs, overlap_map, overlap_graph = overlap_all_pairs(reads, 30)
    print(len(overlap_map))

    print(len(overlap_graph))

In [185]:
def main():
    test1()
    print("All tests completed successfully")
    genome = readGenome('chr1.GRCh38.excerpt.fasta')
    question1(genome)
    question2(genome)

    test2()
    example1()
    example2()
    print("All example tests completed successfully")

    reads, qualities = readFastq('ERR266411_1.for_asm.fastq')
    question3and4(reads)
    print("All done")

In [196]:
question3and4(reads)

904746
7161


In [197]:
# implementing the shortest common superstring

In [14]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching 
        a prefix of 'b' that is at least 'min_length' characters long.  
        If no such overlap exists, return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [23]:
def scs(ss):
    """ Returns shortest common superstring of given strings, which must be the same length """
    import itertools
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

In [29]:
inputStrings = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
shortestCommonSuperstring = scs(inputStrings) #== 'CCTTGGATTGC'

In [30]:
print(len(shortestCommonSuperstring))

11


In [35]:
shortestList = scs_list( ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT'] )

In [36]:
print(len(shortestList))

4


In [18]:
def scs_list(ss):
    """ Returns the alphebeticaly sorted list of shortest common superstrings of given strings, which must be the same length """
    import itertools
    shortest_sup = []
    shortest_length = 0
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_length == 0:
            shortest_sup.append(sup)
            shortest_length = len(sup)
        elif len(sup) < shortest_length:
            shortest_length = len(sup)
            shortest_sup = [sup]
        elif len(sup) == shortest_length:
            shortest_sup.append(sup)
        else:
            #simply ignore and move on
            None
    return sorted(shortest_sup)

In [21]:
scs_list(strings)

['CCTTGGATT']

## 4.2 greedy scs

In [201]:
# implementing greedy shortest common superstring

In [202]:
def pick_maximal_overlap(reads,k):
    reada, readb = None, None
    best_olen = 0
    for a,b in itertools.permutations(reads,2):
        olen = overlap(a,b,min_length=k)
        if olen > best_olen:
            reada, readb = a, b
            best_olen = olen
    return reada, readb, best_olen

In [203]:
def greedy_scs(reads, k):
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads,k)
    return ''.join(reads)

In [205]:
greedy_scs(['ABC', 'BCA', 'CAB'], 2)

'CABCA'

In [206]:
greedy_scs(['ABCD', 'CDBC', 'BCDA'], 1)

'CDBCABCDA'

In [207]:
scs(['ABCD', 'CDBC', 'BCDA'])

'ABCDBCDA'

## 4.3 DeBruijn

In [1]:
# building a de bruikn graph
def de_bruijn_ize(st, k):
    edges = []
    nodes = set()
    for i in range(len(st) - k +1):
        edges.append((st[i:i+k-1], st[i+1:i+k]))
        nodes.add(st[i:i+k-1])
        nodes.add(st[i+1:i+k])
    return nodes, edges

In [2]:
nodes, edges = de_bruijn_ize('ACGCGTCG', 3)

In [3]:
print(nodes)

{'GT', 'CG', 'TC', 'AC', 'GC'}


In [4]:
print(edges)

[('AC', 'CG'), ('CG', 'GC'), ('GC', 'CG'), ('CG', 'GT'), ('GT', 'TC'), ('TC', 'CG')]


In [5]:
def visualize_de_bruijn(st, k):
    nodes, edges = de_bruijn_ize(st,k)
    dot_str = 'digraph "DeBruijn graph" {\n'
    for node in nodes:
        dot_str += ' %s [label="%s"] ;\n' % (node, node)
    for src, dst in edges:
        dot_str += ' %s -> %s ;\n' % (src, dst)
    return dot_str + '}\n'

In [8]:
from logging import info, error
from subprocess import Popen, PIPE

from IPython.display import display, SVG
from IPython.core.magic import Magics
from IPython.core.magic import line_cell_magic
from IPython.core.magic import line_magic
from IPython.core.magic import magics_class

In [9]:
%load_ext gvmagic

ModuleNotFoundError: No module named 'gvmagic'

In [12]:
dbg = visualize_de_bruijn('ACGCGTCG', 3)

In [13]:
dbg

'digraph "DeBruijn graph" {\n GT [label="GT"] ;\n CG [label="CG"] ;\n TC [label="TC"] ;\n AC [label="AC"] ;\n GC [label="GC"] ;\n AC -> CG ;\n CG -> GC ;\n GC -> CG ;\n CG -> GT ;\n GT -> TC ;\n TC -> CG ;\n}\n'

In [37]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq

--2020-12-09 15:12:25--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ads1_week4_reads.fq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.226.211.135, 13.226.211.176, 13.226.211.37, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.226.211.135|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395781 (387K) [video/m2ts]
Saving to: ‘ads1_week4_reads.fq’

ads1_week4_reads.fq 100%[===================>] 386.50K  --.-KB/s    in 0.1s    

2020-12-09 15:12:26 (2.59 MB/s) - ‘ads1_week4_reads.fq’ saved [395781/395781]



In [38]:
overlap_cache = {}

In [39]:
def pick_maximal_overlap(reads, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    import itertools
    reada, readb = None, None
    best_olen = 0

    for a, b in itertools.permutations(reads, 2):
        
        if (a,b) in overlap_cache.keys() and overlap_cache[(a,b)] >= k:
            overlap_len = overlap_cache[(a,b)]
            #print("overlap_cache hit for (a,b):", (a,b), "min was", k, "result was:", overlap_len, "cache size:", len(overlap_cache))
            return a, b, overlap_len
        else:
            olen = overlap(a, b, min_length=k)
            #olen = overlap(a, b)
            if olen > best_olen:
                reada, readb = a, b
                best_olen = olen
                overlap_cache[(a,b)]=best_olen

    return reada, readb, best_olen


In [40]:
def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    lenCacheBefore = len(overlap_cache)
    read_a, read_b, olen = pick_maximal_overlap(reads, k)

    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    
    lenCacheAfter = len(overlap_cache)
    #print("Cached {0} items during this pass".format(lenCacheAfter-lenCacheBefore))

    return ''.join(reads)

In [45]:
reads, qualities = readFastq('ads1_week4_reads.fq')

In [46]:
from datetime import datetime

In [ ]:
for i in range (30, 100):
        result = greedy_scs(list(reads), i) #we make a copy of the reads as the greedy_scs modifies the list
        print("Found result which is {0} bases long for k={1}".format(len(result), i))
        
        # Hint: the virus genome you are assembling is exactly 15,894 bases long
        #assert len(result) == 15894
        if len(result) == 15894:
            print(result.count('A'))
            print(result.count('T'))

In [ ]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists, return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

def scs(ss):
    """ Returns shortest common superstring of given strings, which must be the same length """
    import itertools
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest

def scs_list(ss):
    """ Returns the alphebeticaly sorted list of shortest common superstrings of given strings, which must be the same length """
    import itertools
    shortest_sup = []
    shortest_length = 0
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_length == 0:
            shortest_sup.append(sup)
            shortest_length = len(sup)
        elif len(sup) < shortest_length:
            shortest_length = len(sup)
            shortest_sup = [sup]
        elif len(sup) == shortest_length:
            shortest_sup.append(sup)
        else:
            #simply ignore and move on
            None
    return sorted(shortest_sup)


def test01():
    """Consider the input strings ABC, BCA, CAB. One shortest common superstring is ABCAB but another is BCABC and another is CABCA."""
    inputStrings = ['ABC', 'BCA', 'CAB']
    shortestCommonSuperstring = scs(inputStrings)
    assert shortestCommonSuperstring in ['ABCAB', 'BCABC', 'CABCA']

def question1():
    """What is the length of the shortest common superstring of the following strings? CCT, CTT, TGC, TGG, GAT, ATT"""
    inputStrings = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
    shortestCommonSuperstring = scs(inputStrings) #== 'CCTTGGATTGC'

    #print("shortest common superstring", shortestCommonSuperstring)
    print("What is the length of the shortest common superstring of the following strings? CCT, CTT, TGC, TGG, GAT, ATT")
    print(len(shortestCommonSuperstring))

def example1():
    strings = ['ABC', 'BCA', 'CAB']
    # Returns just one shortest superstring
    assert scs(strings) == 'ABCAB'
    # Returns list of all superstrings that are tied for shorest
    shortestList = scs_list(strings)
    assert shortestList == ['ABCAB', 'BCABC', 'CABCA'], 'found ' + str(shortestList)

def example2():
    ##from pprint import pprint
    strings = ['GAT', 'TAG', 'TCG', 'TGC', 'AAT', 'ATA']
    # Returns just one shortest superstring
    assert scs(strings) == 'TCGATGCAATAG'
    # Returns list of all superstrings that are tied for shorest
    shortestList = scs_list(strings)
    ##pprint(shortestList)

    assert shortestList ==  ['AATAGATCGTGC',
                             'AATAGATGCTCG',
                             'AATAGTCGATGC',
                             'AATCGATAGTGC',
                             'AATGCTCGATAG',
                             'TCGAATAGATGC',
                             'TCGATAGAATGC',
                             'TCGATGCAATAG',
                             'TGCAATAGATCG',
                             'TGCAATCGATAG'], 'found ' + str(shortestList)



def question2():
    """How many different shortest common superstrings are there for the input strings given in the previous question?
    Hint 1: You can modify the scs function to keep track of this."""
    #from pprint import pprint
    shortestList = scs_list( ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT'] )
    #pprint(shortestList)
    print("How many different shortest common superstrings are there for the input strings given in the previous question?")
    print(len(shortestList))


def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            #All the reads are the same length (100 bases) 
            assert len(seq) == 100
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

overlap_cache = {} # 

#copied from: http://nbviewer.ipython.org/github/Benlangmead/ads1-notebooks/blob/master/4.02_GreedySCS.ipynb
def pick_maximal_overlap(reads, k):
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k.  Returns
        overlap length 0 if there are no such overlaps."""
    import itertools
    reada, readb = None, None
    best_olen = 0

    for a, b in itertools.permutations(reads, 2):
        
        if (a,b) in overlap_cache.keys() and overlap_cache[(a,b)] >= k:
            overlap_len = overlap_cache[(a,b)]
            #print("overlap_cache hit for (a,b):", (a,b), "min was", k, "result was:", overlap_len, "cache size:", len(overlap_cache))
            return a, b, overlap_len
        else:
            olen = overlap(a, b, min_length=k)
            #olen = overlap(a, b)
            if olen > best_olen:
                reada, readb = a, b
                best_olen = olen
                overlap_cache[(a,b)]=best_olen

    return reada, readb, best_olen

#copied from: http://nbviewer.ipython.org/github/Benlangmead/ads1-notebooks/blob/master/4.02_GreedySCS.ipynb
def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    lenCacheBefore = len(overlap_cache)
    read_a, read_b, olen = pick_maximal_overlap(reads, k)

    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    
    lenCacheAfter = len(overlap_cache)
    #print("Cached {0} items during this pass".format(lenCacheAfter-lenCacheBefore))

    return ''.join(reads)

def validated_greedy_scs():
    res1 = greedy_scs(['ABC', 'BCA', 'CAB'], 2)
    #print('res1: ', res1)
    assert res1 == 'CABCA'

    res2 = greedy_scs(['ABCD', 'CDBC', 'BCDA'], 1)
    #print('res2: ', res2)
    assert res2 == 'CDBCABCDA'

def question3and4():
    from datetime import datetime
    
    reads, qualities = readFastq('ads1_week4_reads.fq')
    #print(len(reads))
    
    for i in range (30, 100):
        print("timestamp: ", datetime.now())
        result = greedy_scs(list(reads), i) #we make a copy of the reads as the greedy_scs modifies the list
        print("Found result which is {0} bases long for k={1}".format( len(result), i ) )
        
        # Hint: the virus genome you are assembling is exactly 15,894 bases long
        #assert len(result) == 15894
        if len(result) == 15894:
            print("Question3: ", result.count('A'))
            print("Question4: ", result.count('T'))
            return

    for i in range (30, 1, -1):
        print("timestamp: ", datetime.now())
        result = greedy_scs(list(reads), i) #we make a copy of the reads as the greedy_scs modifies the list
        print("Found result which is {0} bases long for k={1}".format( len(result), i ) )
        
        # Hint: the virus genome you are assembling is exactly 15,894 bases long
        #assert len(result) == 15894
        if len(result) == 15894:
            print("Question3: ", result.count('A'))
            print("Question4: ", result.count('T'))
            return

def main():
    test01()
    question1()
    example1()
    example2()
    question2()
    validated_greedy_scs()
    question3and4()
    

if __name__ == '__main__':
    main()

In [ ]:
def question3and4():
    from datetime import datetime
    
    reads, qualities = readFastq('ads1_week4_reads.fq')
    #print(len(reads))
    
    for i in range (30, 100):
        print("timestamp: ", datetime.now())
        result = greedy_scs(list(reads), i) #we make a copy of the reads as the greedy_scs modifies the list
        print("Found result which is {0} bases long for k={1}".format( len(result), i ) )
        
        # Hint: the virus genome you are assembling is exactly 15,894 bases long
        #assert len(result) == 15894
        if len(result) == 15894:
            print("Question3: ", result.count('A'))
            print("Question4: ", result.count('T'))
            return

    for i in range (30, 1, -1):
        print("timestamp: ", datetime.now())
        result = greedy_scs(list(reads), i) #we make a copy of the reads as the greedy_scs modifies the list
        print("Found result which is {0} bases long for k={1}".format( len(result), i ) )
        
        # Hint: the virus genome you are assembling is exactly 15,894 bases long
        #assert len(result) == 15894
        if len(result) == 15894:
            print("Question3: ", result.count('A'))
            print("Question4: ", result.count('T'))
            return